# Drilling down into 4-degree scale statistics of the GEOS-5 Nature Run

The G5NR experiment is described at http://g5nr.nccs.nasa.gov/<br>
Its time span is about 2 years, mid-May to mid-May, with SST from particular years 2005-2007.<br><br>
This software is from https://github.com/brianmapes/CHAD_G5NR <br>
Questions? Brian Mapes ([mapes@miami.edu](mailto:mapes@miami.edu))<br>

Original author was Matthew Niznik (a postdoc 2015-2016), who wrote the more general package:<br>
https://github.com/matthewniznik/ClickHist/wiki<br>
http://matthewniznik.com/research-projects/clickhist<br>

## Set a name for this session. All outputs will go in that directory. 

In [1]:
sessionName = 'Atl_Precip_SKEdot_Summer2005'

### What two 4-degree variables do you want to scatterplot and pick your cases from?

*The units and ranges are hardwired into the module housekeeping_G5NR.py. So you have to pick from a list of options which are set there (case sensitive). The 4 degree dataset has many more quantities whose scatterplot ranges could be set, and then they could be used. See its header for the names of those available variables. Here, TEEF is Total Eddy Enthalpy Flux at 500mb, good for finding deep convection especially in dry 500mb environments. SKEdot is the change in column-integrated shear kinetic energy by eddy momentum flux. HMV is horizontal momentum variance, good for finding tropical storm centers.* <br>
Precip, W500, wPuP, TEEF, SKEDot, HMV

In [2]:
var1Name = 'Precip'
var2Name = 'SKEDot'

### What space-time subset are you interested in exploring? 

Longitude: 0 through 360 (Degrees East), don't try to cross the Prime Meridian...<br>
Latitude: -90 through 90 (Degrees North)<br>
Time: indices of the 18200 hours in this dataset (data starts on nominal "2005-05-16", but recall it is just a simulation).

In [3]:
lonLow =  360. - 70. # Atlantic 70W-0W
lonHigh = 360. - 0. 
latLow = -10.0       # North Tropical belt
latHigh = 20.0

#These are indices in the range [0, 18200 or so], a simple range limiter to speed up data reading
timelimit1 = 0
timelimit2 = 24*30*4 # 4 months

### Which quicklooks from the [online G5NR repository](http://g5nr.nccs.nasa.gov/images/) would you like your notebooks to contain?

**Options:** 'cloudsir', 'cloudsvis', 'cyclones', 'storms', 'temperature', 'tropical', 'water', 'winds'
<br>*Must be a list, even if only one is chosen*

In [4]:
imageVar = ['cloudsir','cyclones','water']

--------
### More options... Hit shift-return again and again to get on with the show!

#### This sets the URL of Brian's hourly 4-degree dataset
These data were derived and rebinned from NASA's 0.5deg set. For faster response times, you could download this 4-degree file locally (it is 6 GB), and then use its pathname as urlToLoad. 
Here is the 6GB file if you want to download it: http://weather.rsmas.miami.edu/repository/entry/show?entryid=synth%3Aeab82de2-d682-4dc0-ba8b-2fac7746d269%3AL2FsbFZhcnNfcjkweDQ1X2hvdXJseS5uYw%3D%3D

In [13]:
urlToLoad = ('http://weather.rsmas.miami.edu/repository/opendap/'+
             'synth:eab82de2-d682-4dc0-ba8b-2fac7746d269:'+
             'L2FsbFZhcnNfcjkweDQ1X2hvdXJseS5uYw==/entry.das')
# Faster from downloaded file
#urlToLoad = ('/Users/bem/Downloads/allVars_r90x45_hourly.nc4')  

#### This is a list of pre-made 'template' IDV bundles, which ClickHist will modify to center on the time and location of YOUR scatterplot clicks. It's nice to have a fast small 'simple' one, and a rich deep 'full' one. 
**Note:** The *first* bundle here will be the one referenced in a script output that allows you to do a batch process that reads in the data and pre-generates images, movies, and a local .zidv file for fast interactive loading. It should probably be the "full" bundle with all the variables you want to study.

In [6]:
bundleInFilenames = ['G5NR_template_full', 'G5NR_template_simple_cloudsir']
bundleOutTags = ['full', 'simple']

#### Set how large you want your IDV bundle of detailed data to be in space and time. Careful, BIG DATA! 
Each of these is calculated as distance from center, so `lonOffset = 1.0` means 2.0° of longitude. `dtFromCenter` needs to be in seconds

In [7]:
lonOffset = 2.0
latOffset = 2.0
dtFromCenter = 2*3600. # in seconds

Specific quantile lines indicated in the scatterplot

In [8]:
quantiles = [0.01,0.1,1,5,95,99,99.9,99.99]

--------
## Now, the action. Hit shift-return again and again to get on with the show!

### Import the necessary modules

*Currently supported graphics backends are Qt4Agg ('qt4') and TK ('tk')*

In [9]:
#%matplotlib tk
%matplotlib qt4
import matplotlib
#matplotlib.use('TkAgg')
#matplotlib.use('Qt4Agg')

from IPython.display import clear_output
import netCDF4 #If this gives an error, in a terminal type conda install netCDF4 or pip netCDF4
import sys

import ClickHist_G5NR as ClickHist
import ClickHistDo_G5NR as ClickHistDo
import housekeeping_G5NR
import numpy as np

#### Scatterplot features (axis labels, joint histogram bin labels, etc.) are here set to values that are hidden in the module `housekeeping_G5NR`

(You can change them in the module if desired, or here directly. Yes, this is inelegant.)

In [10]:
lonValueName = housekeeping_G5NR.lonValueName
latValueName = housekeeping_G5NR.latValueName
timeValueName = housekeeping_G5NR.timeValueName
startDatetime = housekeeping_G5NR.startDatetime

var1Edges = housekeeping_G5NR.binOptions[var1Name]
var2Edges = housekeeping_G5NR.binOptions[var2Name]

var1FmtStr = housekeeping_G5NR.fmtStrOptions[var1Name]
var2FmtStr = housekeeping_G5NR.fmtStrOptions[var2Name]

var1ValueName = housekeeping_G5NR.valueNameOptions[var1Name]
var2ValueName = housekeeping_G5NR.valueNameOptions[var2Name]

var1Units = housekeeping_G5NR.varUnitOptions[var1Name]
var2Units = housekeeping_G5NR.varUnitOptions[var2Name]
metadata_UD = (var1Name+' vs '+var2Name+': '+
               str(lonLow)+' to '+str(lonHigh)+' E, '+
               str(latLow)+' to '+str(latHigh)+' N')

var1ValueMult = housekeeping_G5NR.varMultOptions[var1Name]
var2ValueMult = housekeeping_G5NR.varMultOptions[var2Name]

## Load the Dataset and grab the coordinate variables

In [14]:
cdfIn = netCDF4.Dataset(urlToLoad,'r')

lonValues = cdfIn.variables[lonValueName][:]
latValues = cdfIn.variables[latValueName][:]
timeValues = cdfIn.variables[timeValueName][timelimit1:timelimit2]*housekeeping_G5NR.timeValueMult

#### By finding the needed index ranges here, we will load only the desired subset of the data.

In [15]:
lowLonInt,highLonInt = housekeeping_G5NR.getIntEdges(lonValues,lonLow,lonHigh)
lowLatInt,highLatInt = housekeeping_G5NR.getIntEdges(latValues,latLow,latHigh)

# How many values are we asking for?
print highLatInt+1-lowLatInt, 'x', highLonInt+1-lowLonInt, 'x', np.size(timeValues)

8 x 18 x 2880


#### *Based on the above, the following variable load command may take some time.*

In [16]:
var1Values = cdfIn.variables[var1ValueName][timelimit1:timelimit2,
                                            lowLatInt:highLatInt+1,
                                            lowLonInt:highLonInt+1]*\
                                            var1ValueMult
var2Values = cdfIn.variables[var2ValueName][timelimit1:timelimit2,
                                            lowLatInt:highLatInt+1,
                                            lowLonInt:highLonInt+1]*\
                                            var2ValueMult

np.shape(var1Values)

(2880, 8, 18)

In [17]:
cdfIn.close()

*(We now subset the longitude and latitude coordinate arrays, since the previous call to getIntEdges needed the full arrays. They were not big, just 1D arrays.)*

In [18]:
lonValues = lonValues[lowLonInt:highLonInt+1]
latValues = latValues[lowLatInt:highLatInt+1]

## Create the interactive scatterplot

*This bug workaround call used to be necessary to make sure the output displays properly. (If interested in the details, see: http://bit.ly/1SsishU)*

In [19]:
#oldsysstdout = sys.stdout
#sys.stdout = housekeeping_G5NR.flushfile(sys.stdout)

### Initialize 'ClickHistDo'

In [20]:
ClickHistDo1 = ClickHistDo.ClickHistDo(lonValues,latValues,
                                       timeValues,startDatetime,
                                       bundleInFilenames,
                                       bundleOutTags,
                                       sessionName,
                                       xVarName=var1Name,
                                       yVarName=var2Name,
                                       lonOffset=lonOffset,
                                       latOffset=latOffset,
                                       dtFromCenter=dtFromCenter,
                                       imageVar=imageVar,
                                       openTab=False)

# Launch the clickable histogram, and let the sampling begin! 
## You will get a separate popup clickable window. 

If you want the textual output stream to be in a separate window, uncomment `%qtconsole`. Otherwise it appears below the next cell.

In [21]:
#%qtconsole
ClickHist1 = ClickHist.ClickHist(var1Edges,var2Edges,
                                 var1Values,var2Values,
                                 xVarName=var1Name,yVarName=var2Name,
                                 xUnits=var1Units,yUnits=var2Units,
                                 xFmtStr=var1FmtStr,
                                 yFmtStr=var2FmtStr,
                                 maxPlottedInBin=housekeeping_G5NR.maxPlottedInBin_UD,
                                 quantiles=quantiles,
                                 metadata=metadata_UD)
ClickHist1.setDo(ClickHistDo1)
ClickHist1.showPlot()

Saving IDV bundle(s)...
2005-06-07 12:30:00
332 E  4 N
X= 64 mm day-1 Y=0.933 W m-2
x%: 99.842 y%: 99.921

Link to cloudsir image: http://g5nr.nccs.nasa.gov/static/naturerun/fimages/CLOUDSIR/Y2005/M06/D07/cloudsir_globe_c1440_NR_BETA9-SNAP_20050607_1230z.png

Link to cyclones image: http://g5nr.nccs.nasa.gov/static/naturerun/fimages/CYCLONES/Y2005/M06/D07/cyclones_globe_c1440_NR_BETA9-SNAP_20050607_1230z.png

Link to water image: http://g5nr.nccs.nasa.gov/static/naturerun/fimages/WATER/Y2005/M06/D07/water_globe_c1440_NR_BETA9-SNAP_20050607_1230z.png

Bundle 'full' Saved!
Bundle 'simple' Saved!

Creating Case Notebook (Precip_quantile_99.842_SKEDot_quantile_99.921_lat_04_lon_332_time_20050607_1230.ipynb)
Copy the template
Adding quicklook images to the notebook
Now looping over  ['cloudsir', 'cyclones', 'water'] ...
 trying  cloudsir ...
appending 0
 trying  cyclones ...
appending 1
 trying  water ...
appending 2
Case Notebook created!


In [ ]:
# from subprocess import call
#call('')

# $IDV_HOME/runIDV Output/Scripts/idvImZIDVOutput_Precip_quantile_....isl'